In [2]:
import pandas as pd
real=pd.read_csv('/content/drive/MyDrive/Data Analysis/Fake News Detection/Datasets/True.csv')
fake=pd.read_csv('/content/drive/MyDrive/Data Analysis/Fake News Detection/Datasets/Fake.csv')

In [3]:
real.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
real['is_fake']=0
fake['is_fake']=1
data=pd.concat([real,fake])
data.head()

,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
def clean_text(text):
  text=text.lower()
  text=re.sub(r'[^\w\s]','',text)
  cleaned_words=[]
  for word in text.split():
    if word not in stop_words:
      cleaned_words.append(lemmatizer.lemmatize(word))
  return ' '.join(cleaned_words)
data['cleaned_text'] = data['text'].apply(clean_text)
print(data['text'].iloc[0])
print(data['cleaned_text'].iloc[0])

WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educati

In [8]:
X=data['cleaned_text']
y=data['is_fake']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# 1. Initialize the TF-IDF Vectorizer
# It will convert all text into a matrix of word importance scores
tfidf_vectorizer = TfidfVectorizer(max_features=10000) # Use top 10,000 most common words

# 2. Fit and transform the training data
print("Running TF-IDF on training data...")
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# 3. Transform the test data
print("Running TF-IDF on test data...")
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 4. Initialize the Logistic Regression model
log_model = LogisticRegression(random_state=42, max_iter=1000) # max_iter for convergence

# 5. Train the model
print("Training Logistic Regression model...")
log_model.fit(X_train_tfidf, y_train)

# 6. Make predictions
y_pred_log = log_model.predict(X_test_tfidf)

# 7. Evaluate the baseline model
print("\n--- TF-IDF + Logistic Regression Baseline Results ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_log):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_log))

Running TF-IDF on training data...
Running TF-IDF on test data...
Training Logistic Regression model...

--- TF-IDF + Logistic Regression Baseline Results ---
Accuracy: 0.9886

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4284
           1       0.99      0.99      0.99      4696

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [10]:
# Check how many real news articles contain "reuters"
data[data['is_fake'] == 0]['text'].str.contains('reuters', case=False).value_counts()

,count
text,
True,21378
False,39


In [11]:
!pip install transformers
!pip install datasets

In [12]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 1. Create a new, clean DataFrame
# We only use 'title' and 'is_fake'
df_bert = data[['title', 'is_fake']].copy()

# 2. Rename 'is_fake' to 'label' which Hugging Face expects
df_bert = df_bert.rename(columns={'is_fake': 'label'})

# 3. Drop any rows with missing titles (if any)
df_bert = df_bert.dropna(subset=['title'])

# 4. Create a standard Train/Test split
train_df, test_df = train_test_split(df_bert,
                                     test_size=0.2,
                                     random_state=42,
                                     stratify=df_bert['label'])

# 5. Convert pandas DataFrames into Hugging Face 'Dataset' objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# 6. Combine them into a 'DatasetDict' (the standard for Hugging Face)
raw_datasets = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

print("Hugging Face Dataset created:")
print(raw_datasets)

Hugging Face Dataset created:
DatasetDict({
    train: Dataset({
        features: ['title', 'label', '__index_level_0__'],
        num_rows: 35918
    })
    test: Dataset({
        features: ['title', 'label', '__index_level_0__'],
        num_rows: 8980
    })
})


In [13]:
from transformers import AutoTokenizer

# 1. Load the DistilBERT tokenizer
# This will download the tokenizer model from Hugging Face
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# 2. Create a function to tokenize our 'title' column
def tokenize_function(examples):
    # This will pad short titles and truncate long ones
    return tokenizer(examples['title'], truncation=True, padding="max_length")

# 3. Apply this function to our entire dataset
# The 'batched=True' makes this run very fast
print("Tokenizing all datasets...")
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# 4. Clean up the dataset
# We don't need the original 'title' or '__index_level_0__' columns anymore
tokenized_datasets = tokenized_datasets.remove_columns(['title', '__index_level_0__'])
tokenized_datasets.set_format("torch") # Format for PyTorch

print("\nTokenizing complete.")
print(tokenized_datasets['train'][0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizing all datasets...


Map:   0%|          | 0/35918 [00:00<?, ? examples/s]

Map:   0%|          | 0/8980 [00:00<?, ? examples/s]


Tokenizing complete.
{'label': tensor(0), 'input_ids': tensor([  101,  2605, 18675,  1057,  1012,  1055,  1012,  2000, 11703,  1012,
         2410,  6465,  2006, 12992,  2075,  2954,  2114,  1059,  1012,  3060,
        17671,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
        

In [14]:
from transformers import AutoModelForSequenceClassification

# Load the DistilBERT model with a sequence classification head
# The 'num_labels=2' tells it to create a new head with 2 outputs (0 for 'Real', 1 for 'Fake')
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [16]:
!pip install --upgrade transformers
!pip install --upgrade evaluate

In [17]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

# --- 1. Set up the Metric Function ---
# (This is the same as before)
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # Get the most likely prediction

    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted") # 'weighted' for f1

    return {
        "accuracy": acc["accuracy"],
        "f1": f1["f1"],
    }

# --- 2. Define Training Arguments ---
# *** THE FIX IS HERE ***
# We are using the OLD parameter name 'eval_strategy'
training_args = TrainingArguments(
    output_dir="bert-fake-news-classifier", # Directory to save the model
    learning_rate=2e-5,
    report_to="none",                          # Standard learning rate for BERT
    per_device_train_batch_size=16,        # How many samples to train on at once
    per_device_eval_batch_size=16,         # How many samples to evaluate on at once
    num_train_epochs=3,                    # 3 epochs is standard for fine-tuning
    weight_decay=0.01,                     # Helps prevent overfitting
    eval_strategy="epoch",                 # <-- THE FIX (changed from evaluation_strategy)
    save_strategy="epoch",                 # This name is usually the same
    load_best_model_at_end=True,           # Will load the best model (by loss) at thed
)

# --- 3. Initialize the Trainer ---
# (Also includes the typo fix for compute_metrics)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # <-- ensures typo is fixed
)

/tmp/ipython-input-3552570929.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.067200,0.051043,0.984298,0.984295
2,0.030800,0.060175,0.986303,0.986307
3,0.008400,0.063557,0.988419,0.988421


TrainOutput(global_step=6735, training_loss=0.042683786308668065, metrics={'train_runtime': 5374.2538, 'train_samples_per_second': 20.05, 'train_steps_per_second': 1.253, 'total_flos': 1.4273892074778624e+16, 'train_loss': 0.042683786308668065, 'epoch': 3.0})

In [20]:
# Define a directory to save your final model
model_save_path = "Fake News Detection"

# 1. Save the model
print(f"Saving model to {model_save_path}...")
trainer.save_model(model_save_path)

# 2. Save the tokenizer
tokenizer.save_pretrained(model_save_path)

print("Model and tokenizer saved successfully.")

Saving model to Fake News Detection...
Model and tokenizer saved successfully.
